In [1]:
!pip install histomicstk

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
print(os.listdir(".."))

['src', 'working', 'input', 'lib']


In [3]:


import histomicstk as htk

import numpy as np
import scipy as sp

import skimage.io
import skimage.measure
import skimage.color

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import seaborn as sns
sns.set(color_codes=True)

import pandas as pd

labels = pd.read_csv("../input/train_labels.csv")
labels = labels.set_index('id')

#Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['image.cmap'] = 'gray'
titlesize = 24

#settings

FOREGROUND_THRESHOLD = 60
# adaptive multi-scale LoG filter parameters
MIN_RADIUS = 10
MAX_RADIUS = 15
#filter out small objects
MIN_NUCLEUS_AREA = 70
# show processed images?
SHOWIMAGES=False
# print an asterisk per processed image?
SHOWSTARS=False

In [4]:
#rescale + gaussian
from skimage.transform import resize

def x4(i):
    return rescale(i, (i.shape[0] * 4, i.shape[1] * 4),
                       anti_aliasing=True)


In [5]:
def process(image):
    
    # read image from file
    im_input = skimage.io.imread(image)[:, :, :3]

    # crop
    im_input = im_input[32:64,32:64,:]
    
    # resize?
    im_input = resize(im_input, (256,256))
    

    #Load standard reference image for normalization
    ref_image_file = ('https://data.kitware.com/api/v1/file/'
                      '57718cc28d777f1ecd8a883c/download')  # L1.png
    ref_image_file =('../input/train/8f1569e1140f8d321ad2600d47f47072f09ca0e0.tif')

    im_reference = skimage.io.imread(ref_image_file)[:, :, :3]

    # get mean and stddev of reference image in lab space
    """Computes the mean and standard deviation of the intensities of each
    channel of the given RGB image in LAB color space. The outputs of this
    function are needed for reinhard color normalization.

    Parameters
    ----------
    im_input : array_like
        An RGB image

    Returns
    -------
    mean_lab : array_like
        A 3-element array containing the mean of each channel of the input RGB
        in LAB color space.

    std_lab : array_like
        A 3-element array containing the standard deviation of each channel
        of the input RGB in LAB color space.

    See Also
    --------
    histomicstk.preprocessing.color_conversion.rgb_to_lab,
    histomicstk.preprocessing.color_conversion.reinhard
    """ 
    
    mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(im_reference)

    # perform Reinhard color normalization: https://www.researchgate.net/publication/272749879_A_Complete_Color_Normalization_Approach_To_Histo-pathology_Images_Using_Color_Cues_Computed_From_Saturation-Weighted_Statistics
    
    im_nmzd = htk.preprocessing.color_normalization.reinhard(im_input, mean_ref, std_ref)
    
    # create stain to color map
    
    stainColorMap = {
        'hematoxylin': [0.65, 0.70, 0.29],
        'eosin':       [0.07, 0.99, 0.11],
        'dab':         [0.27, 0.57, 0.78],
        'null':        [0.0, 0.0, 0.0]
    }

    # specify stains of input image
    stain_1 = 'hematoxylin'   # nuclei stain
    stain_2 = 'eosin'         # cytoplasm stain
    stain_3 = 'null'          # set to null of input contains only two stains

    # create stain matrix
    W = np.array([stainColorMap[stain_1], 
                  stainColorMap[stain_2], 
                  stainColorMap[stain_3]]).T

    # perform standard color deconvolution, splitting the image in one channel per color of LAB space
    # https://digitalslidearchive.github.io/HistomicsTK/examples/color-deconvolution.html
    im_stains = htk.preprocessing.color_deconvolution.color_deconvolution(im_nmzd, W).Stains

    # get nuclei/hematoxylin channel
    im_nuclei_stain = im_stains[:, :, 0]

    # segment foreground
    foreground_threshold = FOREGROUND_THRESHOLD

    im_fgnd_mask = sp.ndimage.morphology.binary_fill_holes(
        im_nuclei_stain < foreground_threshold)

    # run adaptive multi-scale LoG filter
    """SCale-adaptive Multiscale Difference-of-Gaussian (DoG) filter for
    nuclei/blob detection.

    Computes the maximal DoG response over a series of scales where in the
    applicable scales at each pixel are constrained to be below an upper-bound
    equal to 2 times the distance to the nearest non-nuclear/background pixel.

    This function uses an approach similar to SIFT interest detection
    where in the scale space between the specified min and max sigma values is
    divided into octaves (scale/sigma is doubled after each octave) and each
    octave is divided into sub-levels. The gaussian images are downsampled by 2
    at the end of each octave to keep the size of convolutional filters small."""
    
    min_radius = MIN_RADIUS
    max_radius = MAX_RADIUS

    im_log_max, im_sigma_max = htk.filters.shape.cdog(
        im_nuclei_stain, im_fgnd_mask,
        sigma_min=min_radius * np.sqrt(2),
        sigma_max=max_radius * np.sqrt(2)
    )

    # detect and segment nuclei using local maximum clustering
    """Local max clustering pixel aggregation for nuclear segmentation.
    Takes as input a constrained log or other filtered nuclear image, a binary
    nuclear mask, and a clustering radius. For each pixel in the nuclear mask,
    the local max is identified. A hierarchy of local maxima is defined, and
    the root nodes used to define the label image."""

    local_max_search_radius = 8

    im_nuclei_seg_mask, seeds, maxima = htk.segmentation.nuclear.max_clustering(
        im_log_max, im_fgnd_mask, local_max_search_radius)

    # filter out small objects
    min_nucleus_area = MIN_NUCLEUS_AREA

    im_nuclei_seg_mask = htk.segmentation.label.area_open(
        im_nuclei_seg_mask, min_nucleus_area).astype(np.int)

    # compute nuclei properties
    objProps = skimage.measure.regionprops(im_nuclei_seg_mask)
    
    #debug verbosity
    # print(f'image {image} segmented, Number of nuclei = {len(objProps)}')
    if SHOWSTARS:
        print ('*', end = "")
    
    if SHOWIMAGES:
          # Display results
        plt.figure(figsize=(20, 10))

        plt.subplot(1, 2, 1)
        plt.imshow(skimage.color.label2rgb(im_nuclei_seg_mask, im_input, bg_label=0), origin='lower')
        plt.title('Nuclei segmentation mask overlay', fontsize=titlesize)

        plt.subplot(1, 2, 2)
        plt.imshow( im_input )
        plt.xlim([0, im_input.shape[1]])
        plt.ylim([0, im_input.shape[0]])
        plt.title('Nuclei bounding boxes', fontsize=titlesize)

        for i in range(len(objProps)):

            c = [objProps[i].centroid[1], objProps[i].centroid[0], 0]
            width = objProps[i].bbox[3] - objProps[i].bbox[1] + 1
            height = objProps[i].bbox[2] - objProps[i].bbox[0] + 1

            cur_bbox = {
                "type":        "rectangle",
                "center":      c,
                "width":       width,
                "height":      height,
            }

            plt.plot(c[0], c[1], 'g+')
            mrect = mpatches.Rectangle([c[0] - 0.5 * width, c[1] - 0.5 * height] , 
                                       width, height, fill=False, ec='g', linewidth=2)
            plt.gca().add_patch(mrect)

    return objProps



In [6]:
def path(image_name):
    return ('../input/train/'+str(image_name)+'.tif')

In [7]:
%%time

import warnings
warnings.filterwarnings('ignore')

# read dataset images from data folder

import pandas as pd

df = pd.read_csv("../input/train_labels.csv")
df.columns=["id","label"]
df["ecc_mean"]=0


#subset rows: uncomment to subset
FROM = 1
TO = 100000
df = df[FROM:TO] 

for index, row in df.iterrows():
    filename, label =row['id'], row['label'] # funzionano
    slide = process(path(filename))
    if not slide:
        continue

    # get eccentricity vector
    eccentricity = []
    for cell in slide:
        eccentricity.append(cell.eccentricity)
    #compute stats : mean, sd, 1quart, 3quart, 1dec, 9dec,...
    if not eccentricity: 
        break
    percentiles = np.percentile(eccentricity, [10,25, 50, 75,90])

    df.loc[index,"ecc_mean"]      = np.mean      (eccentricity)
    df.loc[index,"ecc_min"]       = min          (eccentricity)
    df.loc[index,"ecc_max"]       = max          (eccentricity)
    df.loc[index,"ecc_sd"]        = np.std       (eccentricity)
    df.loc[index,"ecc_skew"]      = sp.stats.skew(eccentricity)
    df.loc[index,"ecc_10pc"]      = percentiles[0]
    df.loc[index,"ecc_1q"]        = percentiles[1]
    df.loc[index,"ecc_median"]    = percentiles[2]
    df.loc[index,"ecc_3q"]        = percentiles[3]
    df.loc[index,"ecc_90pc"]      = percentiles[4]
    del(eccentricity)

    # get area vector
    area = []
          
    area.append(cell.area)
    #compute stats : mean, sd, 1quart, 3quart, 1dec, 9dec,...
    if not area:
        break
    percentiles = np.percentile(area, [10,25, 50, 75,90])

    df.loc[index,"area_mean"]      = np.mean      (area)
    df.loc[index,"area_min"]       = min          (area)
    df.loc[index,"area_max"]       = max          (area)
    df.loc[index,"area_sd"]        = np.std       (area)
    df.loc[index,"area_skew"]      = sp.stats.skew(area)
    df.loc[index,"area_10pc"]      = percentiles[0]
    df.loc[index,"area_1q"]        = percentiles[1]
    df.loc[index,"area_median"]    = percentiles[2]
    df.loc[index,"area_3q"]        = percentiles[3]
    df.loc[index,"area_90pc"]      = percentiles[4]
    del(area)

    # get solidity vector
    conv = []
    for cell in slide:
        conv.append(cell.area/cell.convex_area)
    #compute stats : mean, sd, 1quart, 3quart, 1dec, 9dec,...
    if not conv:
        break
    percentiles = np.percentile(conv, [10,25, 50, 75,90])

    df.loc[index,"solidity_mean"]      = np.mean      (conv)
    df.loc[index,"solidity_min"]       = min          (conv)
    df.loc[index,"solidity_max"]       = max          (conv)
    df.loc[index,"solidity_sd"]        = np.std       (conv)
    df.loc[index,"solidity_skew"]      = sp.stats.skew(conv)
    df.loc[index,"solidity_10pc"]      = percentiles[0]
    df.loc[index,"solidity_1q"]        = percentiles[1]
    df.loc[index,"solidity_median"]    = percentiles[2]
    df.loc[index,"solidity_3q"]        = percentiles[3]
    df.loc[index,"solidity_90pc"]      = percentiles[4]
    del(conv)
    
    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#print(df.head())

df.to_csv("stats2.csv")


CPU times: user 5h 53min 18s, sys: 4h 32min 27s, total: 10h 25min 45s
Wall time: 5h 18min 17s


In [8]:
# Download CSV with all stats:'''

from IPython.display import HTML
import base64
'''
# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)

def create_download_link(df, title = "Download CSV file", filename = "stats.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# create a link to download the dataframe
create_download_link(df)
'''

'\n# function that takes in a dataframe and creates a text link to  \n# download it (will only work for files < 2MB or so)\n\ndef create_download_link(df, title = "Download CSV file", filename = "stats.csv"):  \n    csv = df.to_csv()\n    b64 = base64.b64encode(csv.encode())\n    payload = b64.decode()\n    html = \'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>\'\n    html = html.format(payload=payload,title=title,filename=filename)\n    return HTML(html)\n\n\n# create a link to download the dataframe\ncreate_download_link(df)\n'